In [2]:
from shapely.geometry import Point, mapping, shape
from shapely.ops import transform
from pyproj import CRS, Transformer
import json

def create_circle_geojson_shapely(lon, lat, radius_meters, output_file):
    # Determine UTM zone for projection
    utm_zone = int((lon + 180) / 6) + 1
    is_northern = lat >= 0

    crs_wgs84 = CRS("EPSG:4326")
    crs_utm = CRS.from_proj4(
        f"+proj=utm +zone={utm_zone} +datum=WGS84 +units=m +{'north' if is_northern else 'south'}"
    )

    # Create transformer functions
    to_utm = Transformer.from_crs(crs_wgs84, crs_utm, always_xy=True).transform
    to_wgs84 = Transformer.from_crs(crs_utm, crs_wgs84, always_xy=True).transform

    # Create buffer around point in UTM (meter units)
    center_utm = transform(to_utm, Point(lon, lat))
    circle_utm = center_utm.buffer(radius_meters, resolution=64)  # resolution = smoothness

    # Reproject back to WGS84 for GeoJSON
    circle_wgs84 = transform(to_wgs84, circle_utm)

    # Create GeoJSON Feature
    feature = {
        "type": "Feature",
        "geometry": mapping(circle_wgs84),
        "properties": {
            "center": [lon, lat],
            "radius_m": radius_meters
        }
    }

    feature_collection = {
        "type": "FeatureCollection",
        "features": [feature]
    }

    # Save to file
    with open(output_file, 'w') as f:
        json.dump(feature_collection, f, indent=2)

    print(f"Circle saved to {output_file}")


In [6]:
#create 1 circle test
create_circle_geojson_shapely(
    lon=-82.400011,
    lat=34.848497,
    radius_meters=1609.34,  # 1 mile
    output_file=r'C:\Users\Joe\Desktop\geojsons\circle1.geojson'
)


Circle saved to C:\Users\Joe\Desktop\geojsons\circle1.geojson


In [18]:
#create 50 circles for 50 mile radius
for i in range(50):
    create_circle_geojson_shapely(
        lon=-82.400011,
        lat=34.848497,
        radius_meters=(1609.34*(i+1)),  # 1 mile
        output_file=rf'C:\Users\Joe\Desktop\geojsons\circle{i+1}.geojson'
    )


Circle saved to C:\Users\Joe\Desktop\geojsons\circle1.geojson
Circle saved to C:\Users\Joe\Desktop\geojsons\circle2.geojson
Circle saved to C:\Users\Joe\Desktop\geojsons\circle3.geojson
Circle saved to C:\Users\Joe\Desktop\geojsons\circle4.geojson
Circle saved to C:\Users\Joe\Desktop\geojsons\circle5.geojson
Circle saved to C:\Users\Joe\Desktop\geojsons\circle6.geojson
Circle saved to C:\Users\Joe\Desktop\geojsons\circle7.geojson
Circle saved to C:\Users\Joe\Desktop\geojsons\circle8.geojson
Circle saved to C:\Users\Joe\Desktop\geojsons\circle9.geojson
Circle saved to C:\Users\Joe\Desktop\geojsons\circle10.geojson
Circle saved to C:\Users\Joe\Desktop\geojsons\circle11.geojson
Circle saved to C:\Users\Joe\Desktop\geojsons\circle12.geojson
Circle saved to C:\Users\Joe\Desktop\geojsons\circle13.geojson
Circle saved to C:\Users\Joe\Desktop\geojsons\circle14.geojson
Circle saved to C:\Users\Joe\Desktop\geojsons\circle15.geojson
Circle saved to C:\Users\Joe\Desktop\geojsons\circle16.geojson
C

In [46]:
#create circles from a json with key value pairs: key is name of town at center and values are latitude and longitude
#create 10 circles for 10 mile radius
def create_10_mile_circles(json_file_path):
    with open(json_file_path) as f:
        areas = json.load(f)
    for key, value in areas.items():
        for i in range(10):
            create_circle_geojson_shapely(
                lon=value[1],
                lat=value[0],
                radius_meters=(1609.34*(i+1)),  # 1 mile
                output_file=rf'C:\Users\Joe\Desktop\geojsons\circle{i+1}{key}.geojson'
            )


In [47]:
#create circles for other towns
create_10_mile_circles(r"C:\Users\Joe\Desktop\geojsons\secondary_areas.JSON")

Circle saved to C:\Users\Joe\Desktop\geojsons\circle1Seneca.geojson
Circle saved to C:\Users\Joe\Desktop\geojsons\circle2Seneca.geojson
Circle saved to C:\Users\Joe\Desktop\geojsons\circle3Seneca.geojson
Circle saved to C:\Users\Joe\Desktop\geojsons\circle4Seneca.geojson
Circle saved to C:\Users\Joe\Desktop\geojsons\circle5Seneca.geojson
Circle saved to C:\Users\Joe\Desktop\geojsons\circle6Seneca.geojson
Circle saved to C:\Users\Joe\Desktop\geojsons\circle7Seneca.geojson
Circle saved to C:\Users\Joe\Desktop\geojsons\circle8Seneca.geojson
Circle saved to C:\Users\Joe\Desktop\geojsons\circle9Seneca.geojson
Circle saved to C:\Users\Joe\Desktop\geojsons\circle10Seneca.geojson
Circle saved to C:\Users\Joe\Desktop\geojsons\circle1Clemson.geojson
Circle saved to C:\Users\Joe\Desktop\geojsons\circle2Clemson.geojson
Circle saved to C:\Users\Joe\Desktop\geojsons\circle3Clemson.geojson
Circle saved to C:\Users\Joe\Desktop\geojsons\circle4Clemson.geojson
Circle saved to C:\Users\Joe\Desktop\geojso

In [92]:
create_circle_geojson_shapely(
                lon=-82.953231,
                lat=34.685709,
                radius_meters=(1609.34*(1)),  # 1 mile
                output_file=rf'C:\Users\Joe\Desktop\geojsons\circle1Seneca.geojson'
            )
create_circle_geojson_shapely(
                lon=-82.953231,
                lat=34.685709,
                radius_meters=(1609.34*(2)),  # 1 mile
                output_file=rf'C:\Users\Joe\Desktop\geojsons\circle2Seneca.geojson'
            )

Circle saved to C:\Users\Joe\Desktop\geojsons\circle1Seneca.geojson
Circle saved to C:\Users\Joe\Desktop\geojsons\circle2Seneca.geojson


In [48]:
#turn circles in other towns into 1 mile radius strips
with open(r"C:\Users\Joe\Desktop\geojsons\secondary_areas.JSON") as f:
    areas = json.load(f)
for key, value in areas.items():
    for i in range(9):
        remove_overlap_with_geojson(rf"C:\Users\Joe\Desktop\geojsons\circle{10-i}{key}.geojson", rf"C:\Users\Joe\Desktop\geojsons\circle{9-i}{key}.geojson")

Overlapping areas removed from C:\Users\Joe\Desktop\geojsons\circle10Seneca.geojson using mask from C:\Users\Joe\Desktop\geojsons\circle9Seneca.geojson.
Overlapping areas removed from C:\Users\Joe\Desktop\geojsons\circle9Seneca.geojson using mask from C:\Users\Joe\Desktop\geojsons\circle8Seneca.geojson.
Overlapping areas removed from C:\Users\Joe\Desktop\geojsons\circle8Seneca.geojson using mask from C:\Users\Joe\Desktop\geojsons\circle7Seneca.geojson.
Overlapping areas removed from C:\Users\Joe\Desktop\geojsons\circle7Seneca.geojson using mask from C:\Users\Joe\Desktop\geojsons\circle6Seneca.geojson.
Overlapping areas removed from C:\Users\Joe\Desktop\geojsons\circle6Seneca.geojson using mask from C:\Users\Joe\Desktop\geojsons\circle5Seneca.geojson.
Overlapping areas removed from C:\Users\Joe\Desktop\geojsons\circle5Seneca.geojson using mask from C:\Users\Joe\Desktop\geojsons\circle4Seneca.geojson.
Overlapping areas removed from C:\Users\Joe\Desktop\geojsons\circle4Seneca.geojson usin

In [33]:
#create circles to remove overlap in multiple circles
def create_removal_circles_10_miles(json_file_path):
    with open(json_file_path) as f:
        areas = json.load(f)
    for key, value in areas.items():
            create_circle_geojson_shapely(
                lon=value[1],
                lat=value[0],
                radius_meters=(1609.34*10),  # 1 mile
                output_file=rf'C:\Users\Joe\Desktop\geojsons\circle{10}{key}removal.geojson'
            )


In [34]:
create_removal_circles_10_miles(r"C:\Users\Joe\Desktop\geojsons\secondary_areas.JSON")

Circle saved to C:\Users\Joe\Desktop\geojsons\circle10Senecaremoval.geojson
Circle saved to C:\Users\Joe\Desktop\geojsons\circle10Clemsonremoval.geojson
Circle saved to C:\Users\Joe\Desktop\geojsons\circle10Andersonremoval.geojson
Circle saved to C:\Users\Joe\Desktop\geojsons\circle10Libertyremoval.geojson
Circle saved to C:\Users\Joe\Desktop\geojsons\circle10Easleyremoval.geojson
Circle saved to C:\Users\Joe\Desktop\geojsons\circle10Piedmontremoval.geojson
Circle saved to C:\Users\Joe\Desktop\geojsons\circle10Fountain_Innremoval.geojson
Circle saved to C:\Users\Joe\Desktop\geojsons\circle10Duncanremoval.geojson


In [35]:
#create removal circle for already completed greenville
create_circle_geojson_shapely(
        lon=-82.400011,
        lat=34.848497,
        radius_meters=(1609.34*(10)),  # 1 mile
        output_file=rf'C:\Users\Joe\Desktop\geojsons\circle{10}Greenvilleremoval.geojson'
    )

Circle saved to C:\Users\Joe\Desktop\geojsons\circle10Greenvilleremoval.geojson


In [50]:
#remove overlap for greenville
with open(r"C:\Users\Joe\Desktop\geojsons\secondary_areas.JSON") as f:
    areas = json.load(f)
for key, value in areas.items():
    for i in range(10):
        remove_overlap_with_geojson(rf"C:\Users\Joe\Desktop\geojsons\circle{10-i}{key}.geojson", rf"C:\Users\Joe\Desktop\geojsons\circle10Greenvilleremoval.geojson")
#remove the overlap from the removal as well so it doesnt delete areas where both removals overlap
    remove_overlap_with_geojson(rf"C:\Users\Joe\Desktop\geojsons\circle10{key}removal.geojson", rf"C:\Users\Joe\Desktop\geojsons\circle10Greenvilleremoval.geojson")
    

Overlapping areas removed from C:\Users\Joe\Desktop\geojsons\circle10Seneca.geojson using mask from C:\Users\Joe\Desktop\geojsons\circle10Greenvilleremoval.geojson.
Overlapping areas removed from C:\Users\Joe\Desktop\geojsons\circle9Seneca.geojson using mask from C:\Users\Joe\Desktop\geojsons\circle10Greenvilleremoval.geojson.
Overlapping areas removed from C:\Users\Joe\Desktop\geojsons\circle8Seneca.geojson using mask from C:\Users\Joe\Desktop\geojsons\circle10Greenvilleremoval.geojson.
Overlapping areas removed from C:\Users\Joe\Desktop\geojsons\circle7Seneca.geojson using mask from C:\Users\Joe\Desktop\geojsons\circle10Greenvilleremoval.geojson.
Overlapping areas removed from C:\Users\Joe\Desktop\geojsons\circle6Seneca.geojson using mask from C:\Users\Joe\Desktop\geojsons\circle10Greenvilleremoval.geojson.
Overlapping areas removed from C:\Users\Joe\Desktop\geojsons\circle5Seneca.geojson using mask from C:\Users\Joe\Desktop\geojsons\circle10Greenvilleremoval.geojson.
Overlapping are

In [51]:
#remove overlap for the rest of the new towns
with open(r"C:\Users\Joe\Desktop\geojsons\secondary_areas.JSON") as f:
    areas = json.load(f)
    areas2=areas
for key, value in areas.items():
    for key2, value2 in areas2.items():
        if key!=key2:
            for i in range(10):
                remove_overlap_with_geojson(rf"C:\Users\Joe\Desktop\geojsons\circle{10-i}{key2}.geojson", rf"C:\Users\Joe\Desktop\geojsons\circle10{key}removal.geojson")
            #remove the overlap from the removal as well so it doesnt delete areas where both removals overlap
            remove_overlap_with_geojson(rf"C:\Users\Joe\Desktop\geojsons\circle10{key2}removal.geojson", rf"C:\Users\Joe\Desktop\geojsons\circle10{key}removal.geojson")
    

Overlapping areas removed from C:\Users\Joe\Desktop\geojsons\circle10Clemson.geojson using mask from C:\Users\Joe\Desktop\geojsons\circle10Senecaremoval.geojson.
Overlapping areas removed from C:\Users\Joe\Desktop\geojsons\circle9Clemson.geojson using mask from C:\Users\Joe\Desktop\geojsons\circle10Senecaremoval.geojson.
Overlapping areas removed from C:\Users\Joe\Desktop\geojsons\circle8Clemson.geojson using mask from C:\Users\Joe\Desktop\geojsons\circle10Senecaremoval.geojson.
Overlapping areas removed from C:\Users\Joe\Desktop\geojsons\circle7Clemson.geojson using mask from C:\Users\Joe\Desktop\geojsons\circle10Senecaremoval.geojson.
Overlapping areas removed from C:\Users\Joe\Desktop\geojsons\circle6Clemson.geojson using mask from C:\Users\Joe\Desktop\geojsons\circle10Senecaremoval.geojson.
Overlapping areas removed from C:\Users\Joe\Desktop\geojsons\circle5Clemson.geojson using mask from C:\Users\Joe\Desktop\geojsons\circle10Senecaremoval.geojson.
Overlapping areas removed from C:

In [84]:
#save areas for new towns as key:value pairs
output = {}
with open(r"C:\Users\Joe\Desktop\geojsons\secondary_areas.JSON") as f:
    areas = json.load(f)
for key, value in areas.items():
    for i in range(10):
        file_path = rf"C:\Users\Joe\Desktop\geojsons\circle{i+1}{key}.geojson"
        new_key = f"{i+1}{key}"
        try:
            output[new_key] = calculate_geojson_area(file_path)
            if output[new_key]==None:
                output[new_key]=0
        except Exception as e:
            print(f"Error processing {file_path}: {e}")
            output[new_key] = 0  # Or handle error appropriately

# Save to JSON file
output_file = r"C:\Users\Joe\Desktop\geojsons\secondary_areas_total_areas.JSON"
with open(output_file, 'w') as f:
    json.dump(output, f, indent=4)

In [86]:
print(calculate_geojson_area(r"C:\Users\Joe\Desktop\geojsons\circle1Clemson.geojson"))
print(None/0)

None


TypeError: unsupported operand type(s) for /: 'NoneType' and 'int'

In [88]:
import json
from shapely.geometry import shape
from shapely.ops import transform
from pyproj import CRS, Transformer


def calculate_geojson_area(geojson_file):
    # Load the GeoJSON
    with open(geojson_file, 'r') as f:
        data = json.load(f)

    total_area_m2 = 0.0

    for feature in data['features']:
        geom = shape(feature['geometry'])

        # Get centroid for local projection (UTM)
        centroid = geom.centroid
        lon, lat = centroid.x, centroid.y

        # Determine UTM zone
        utm_zone = int((lon + 180) / 6) + 1
        is_northern = lat >= 0

        crs_wgs84 = CRS("EPSG:4326")
        crs_utm = CRS.from_proj4(
            f"+proj=utm +zone={utm_zone} +datum=WGS84 +units=m +{'north' if is_northern else 'south'}"
        )

        # Create projection function
        project = Transformer.from_crs(crs_wgs84, crs_utm, always_xy=True).transform

        # Project geometry to meters (UTM) and calculate area
        geom_projected = transform(project, geom)
        total_area_m2 += geom_projected.area
        if total_area_m2==None:
            return 0
        else:
            return total_area_m2  # square meters


In [91]:
with open(r"C:\Users\Joe\Desktop\geojsons\secondary_areas_total_areas.JSON") as f:
    areas = json.load(f)

for key, value in areas.items():

    print(''.join(filter(str.isalpha, key)))

Seneca
Seneca
Seneca
Seneca
Seneca
Seneca
Seneca
Seneca
Seneca
Seneca
Clemson
Clemson
Clemson
Clemson
Clemson
Clemson
Clemson
Clemson
Clemson
Clemson
Anderson
Anderson
Anderson
Anderson
Anderson
Anderson
Anderson
Anderson
Anderson
Anderson
Liberty
Liberty
Liberty
Liberty
Liberty
Liberty
Liberty
Liberty
Liberty
Liberty
Easley
Easley
Easley
Easley
Easley
Easley
Easley
Easley
Easley
Easley
Piedmont
Piedmont
Piedmont
Piedmont
Piedmont
Piedmont
Piedmont
Piedmont
Piedmont
Piedmont
FountainInn
FountainInn
FountainInn
FountainInn
FountainInn
FountainInn
FountainInn
FountainInn
FountainInn
FountainInn
Duncan
Duncan
Duncan
Duncan
Duncan
Duncan
Duncan
Duncan
Duncan
Duncan


In [13]:
import json
import random
from shapely.geometry import shape, Point


def get_random_point_within_geojson(geojson_file):
    # Load the GeoJSON
    with open(geojson_file, 'r') as f:
        data = json.load(f)

    # Combine all geometries
    combined_geom = shape(data['features'][0]['geometry'])
    for feature in data['features'][1:]:
        combined_geom = combined_geom.union(shape(feature['geometry']))

    # Bounding box for sampling
    minx, miny, maxx, maxy = combined_geom.bounds

    # Try generating a point inside the polygon
    for _ in range(10000):
        lat = random.uniform(minx, maxx)
        lon = random.uniform(miny, maxy)
        point = Point(lat, lon)
        if combined_geom.contains(point):
            return [point.y, point.x]  # [lat,lon]

    raise ValueError("Couldn't find a point inside the GeoJSON geometry.")


In [16]:
for _ in range(10):
    print(get_random_point_within_geojson(r"C:\Users\Joe\Desktop\geojsons\circle1.geojson"))

[34.85284650550471, -82.403691173487]
34.84003204827363
[34.84792251598909, -82.4022808121476]
34.86070028221719
[34.837721358729254, -82.400057020711]
34.84882147325285
[34.849581060257485, -82.40416596567839]
34.84360061317873
[34.848836183685606, -82.38279946689109]
34.8464423292678
[34.846196446743555, -82.4123028329081]
34.835263595439386
[34.84681109153615, -82.4027555401648]
34.852834536755196
[34.84237935659722, -82.40215882789761]
34.845505229559166
[34.84366292946084, -82.41207609538395]
34.840341443696595
[34.85626525100694, -82.39354950414919]
34.83617739188698


In [18]:
#remove area from geojson using google data, create new file
from shapely.geometry import shape, Point, mapping
from shapely.ops import transform
from pyproj import CRS, Transformer
import json

def remove_overlap_with_circle(input_geojson, lat, lon, radius_meters, output_geojson):
    # Determine UTM zone for projection
    utm_zone = int((lon + 180) / 6) + 1
    is_northern = lat >= 0

    crs_wgs84 = CRS("EPSG:4326")
    crs_utm = CRS.from_proj4(
        f"+proj=utm +zone={utm_zone} +datum=WGS84 +units=m +{'north' if is_northern else 'south'}"
    )

    # Define transformation functions
    to_utm = Transformer.from_crs(crs_wgs84, crs_utm, always_xy=True).transform
    to_wgs84 = Transformer.from_crs(crs_utm, crs_wgs84, always_xy=True).transform

    # Create the circle in UTM
    center_utm = transform(to_utm, Point(lon, lat))
    circle_utm = center_utm.buffer(radius_meters, resolution=64)

    # Load the input GeoJSON
    with open(input_geojson, 'r') as f:
        data = json.load(f)

    new_features = []
    for feature in data.get("features", []):
        geom = shape(feature["geometry"])
        geom_utm = transform(to_utm, geom)

        # Subtract the circle if it overlaps
        if geom_utm.intersects(circle_utm):
            difference = geom_utm.difference(circle_utm)
            if not difference.is_empty:
                new_geom = transform(to_wgs84, difference)
                new_feature = {
                    "type": "Feature",
                    "geometry": mapping(new_geom),
                    "properties": feature.get("properties", {})
                }
                new_features.append(new_feature)
        else:
            # No intersection, keep original
            new_features.append(feature)

    # Save updated GeoJSON
    updated_geojson = {
        "type": "FeatureCollection",
        "features": new_features
    }

    with open(output_geojson, 'w') as f:
        json.dump(updated_geojson, f, indent=2)

    print(f"Output saved to {output_geojson}")


In [19]:
randomPoint=get_random_point_within_geojson(r"C:\Users\Joe\Desktop\geojsons\circle1.geojson")
remove_overlap_with_circle(r"C:\Users\Joe\Desktop\geojsons\circle1.geojson", randomPoint[0],randomPoint[1], 100, r"C:\Users\Joe\Desktop\geojsons\circle1_revised.geojson")

Output saved to C:\Users\Joe\Desktop\geojsons\circle1_revised.geojson


In [20]:
#remove area from geojson using google data, overwrite file
from shapely.geometry import shape, Point, mapping
from shapely.ops import transform
from pyproj import CRS, Transformer
import json

def remove_overlap_with_circle(input_geojson, lat, lon, radius_meters):
    # Determine UTM zone
    utm_zone = int((lon + 180) / 6) + 1
    is_northern = lat >= 0

    crs_wgs84 = CRS("EPSG:4326")
    crs_utm = CRS.from_proj4(
        f"+proj=utm +zone={utm_zone} +datum=WGS84 +units=m +{'north' if is_northern else 'south'}"
    )

    # Set up projection transformers
    to_utm = Transformer.from_crs(crs_wgs84, crs_utm, always_xy=True).transform
    to_wgs84 = Transformer.from_crs(crs_utm, crs_wgs84, always_xy=True).transform

    # Create circle in UTM
    center_utm = transform(to_utm, Point(lon, lat))
    circle_utm = center_utm.buffer(radius_meters, resolution=64)

    # Load input GeoJSON
    with open(input_geojson, 'r') as f:
        data = json.load(f)

    updated_features = []
    for feature in data.get("features", []):
        geom = shape(feature["geometry"])
        geom_utm = transform(to_utm, geom)

        # Remove overlap if present
        if geom_utm.intersects(circle_utm):
            diff = geom_utm.difference(circle_utm)
            if not diff.is_empty:
                new_geom = transform(to_wgs84, diff)
                updated_features.append({
                    "type": "Feature",
                    "geometry": mapping(new_geom),
                    "properties": feature.get("properties", {})
                })
        else:
            updated_features.append(feature)

    # Overwrite the input file
    updated_geojson = {
        "type": "FeatureCollection",
        "features": updated_features
    }

    with open(input_geojson, 'w') as f:
        json.dump(updated_geojson, f, indent=2)

    print(f"Overlapping areas removed and {input_geojson} has been updated.")


In [21]:
randomPoint=get_random_point_within_geojson(r"C:\Users\Joe\Desktop\geojsons\circle1_revised.geojson")
remove_overlap_with_circle(r"C:\Users\Joe\Desktop\geojsons\circle1_revised.geojson", randomPoint[0],randomPoint[1], 100)

Overlapping areas removed and C:\Users\Joe\Desktop\geojsons\circle1_revised.geojson has been updated.


In [23]:
#cuts out iner radius to create 1 mile radius circles, updates original geojson
from shapely.geometry import shape, mapping
from shapely.ops import unary_union
import json

def remove_overlap_with_geojson(input_geojson, remove_geojson):
    # Load both GeoJSON files
    with open(input_geojson, 'r') as f:
        input_data = json.load(f)

    with open(remove_geojson, 'r') as f:
        circle_data = json.load(f)

    # Combine all geometries from the circle_geojson into a single unioned shape
    mask_shapes = [shape(f["geometry"]) for f in circle_data.get("features", [])]
    mask_union = unary_union(mask_shapes)

    updated_features = []
    for feature in input_data.get("features", []):
        geom = shape(feature["geometry"])

        # Subtract the mask if it intersects
        if geom.intersects(mask_union):
            diff = geom.difference(mask_union)
            if not diff.is_empty:
                updated_features.append({
                    "type": "Feature",
                    "geometry": mapping(diff),
                    "properties": feature.get("properties", {})
                })
        else:
            # No overlap, keep the original
            updated_features.append(feature)

    # Overwrite the input GeoJSON with updated features
    updated_geojson = {
        "type": "FeatureCollection",
        "features": updated_features
    }

    with open(input_geojson, 'w') as f:
        json.dump(updated_geojson, f, indent=2)

    print(f"Overlapping areas removed from {input_geojson} using mask from {remove_geojson}.")


In [20]:
for i in range(49):
    remove_overlap_with_geojson(rf"C:\Users\Joe\Desktop\geojsons\circle{50-i}.geojson", rf"C:\Users\Joe\Desktop\geojsons\circle{49-i}.geojson")

Overlapping areas removed from C:\Users\Joe\Desktop\geojsons\circle50.geojson using mask from C:\Users\Joe\Desktop\geojsons\circle49.geojson.
Overlapping areas removed from C:\Users\Joe\Desktop\geojsons\circle49.geojson using mask from C:\Users\Joe\Desktop\geojsons\circle48.geojson.
Overlapping areas removed from C:\Users\Joe\Desktop\geojsons\circle48.geojson using mask from C:\Users\Joe\Desktop\geojsons\circle47.geojson.
Overlapping areas removed from C:\Users\Joe\Desktop\geojsons\circle47.geojson using mask from C:\Users\Joe\Desktop\geojsons\circle46.geojson.
Overlapping areas removed from C:\Users\Joe\Desktop\geojsons\circle46.geojson using mask from C:\Users\Joe\Desktop\geojsons\circle45.geojson.
Overlapping areas removed from C:\Users\Joe\Desktop\geojsons\circle45.geojson using mask from C:\Users\Joe\Desktop\geojsons\circle44.geojson.
Overlapping areas removed from C:\Users\Joe\Desktop\geojsons\circle44.geojson using mask from C:\Users\Joe\Desktop\geojsons\circle43.geojson.
Overla

In [10]:
#cuts out iner radius to create 1 mile radius circles, creates new file
import os
import json
from shapely.geometry import shape, mapping
from shapely.ops import unary_union

def remove_overlap_with_geojson(input_geojson, remove_geojson):
    # Load input and removal GeoJSON files
    with open(input_geojson, 'r') as f:
        input_data = json.load(f)

    with open(remove_geojson, 'r') as f:
        remove_data = json.load(f)

    # Union of all removal geometries
    mask_shapes = [shape(f["geometry"]) for f in remove_data.get("features", [])]
    mask_union = unary_union(mask_shapes)

    updated_features = []
    for feature in input_data.get("features", []):
        geom = shape(feature["geometry"])

        if geom.intersects(mask_union):
            diff = geom.difference(mask_union)
            if not diff.is_empty:
                updated_features.append({
                    "type": "Feature",
                    "geometry": mapping(diff),
                    "properties": feature.get("properties", {})
                })
        else:
            updated_features.append(feature)

    # Create new filename by appending "_center_removed" before extension
    base, ext = os.path.splitext(input_geojson)
    output_geojson = f"{base}_center_removed{ext}"

    with open(output_geojson, 'w') as f:
        json.dump({
            "type": "FeatureCollection",
            "features": updated_features
        }, f, indent=2)

    print(f"Output written to {output_geojson} (overlapping areas removed).")


In [12]:
remove_overlap_with_geojson(r"C:\Users\Joe\Desktop\geojsons\circle2.geojson", r"C:\Users\Joe\Desktop\geojsons\circle1.geojson")

Output written to C:\Users\Joe\Desktop\geojsons\circle2_center_removed.geojson (overlapping areas removed).


In [4]:
#create a json file with key value pairs of original circles to compare and determine what percentage of the area you have covered
import os
import json
from shapely.geometry import shape
from shapely.ops import transform
from pyproj import CRS, Transformer

# Directory and output path
geojson_dir = r"C:\Users\Joe\Desktop\geojsons"
output_json = r"C:\Users\Joe\Desktop\geojsons\geojson_total_areas.json"

# Collect and calculate areas
area_results = {}
for i in range(1, 51):
    filename = f"circle{i}.geojson"
    filepath = os.path.join(geojson_dir, filename)
    if os.path.exists(filepath):
        area = calculate_geojson_area(filepath)
        area_results[filename] = area

# Save as JSON
with open(output_json, 'w') as f:
    json.dump(area_results, f, indent=2)

print(f"Saved area results to {output_json}")


Saved area results to C:\Users\Joe\Desktop\geojsons\geojson_total_areas.json


In [7]:
#import the total area key value pairs from previous cell
import json

with open(r"C:\Users\Joe\Desktop\geojsons\geojson_total_areas.json") as f:
    areas = json.load(f)

print(areas["circle1.geojson"])
print(calculate_geojson_area(r"C:\Users\Joe\Desktop\geojsons\circle1.geojson"))

8135830.288069832
8135830.288069832
